In [ ]:
# Cell 1: Download the dataset from Kaggle using kagglehub
import kagglehub

# Download latest version
path = kagglehub.dataset_download("dylanjcastillo/7k-books-with-metadata")

print("Path to dataset files:", path)

In [ ]:
import pandas as pd

path = r"C:\Users\ravis\.cache\kagglehub\datasets\dylanjcastillo\7k-books-with-metadata\versions\3\books.csv"

books = pd.read_csv(path)

In [ ]:
books.head()

In [ ]:
# Cell 4: Add columns for missing descriptions and calculate the age of each book
import numpy as np

books["missing_description"] = np.where(books["description"].isna(), 1, 0)
books["age_of_book"] = 2023 - books["published_year"]

In [ ]:
# Cell 5: Compute and visualize the Spearman correlation matrix for selected columns
columns = ['num_pages','age_of_book', 'missing_description', 'average_rating']

correlation_matrix = books[columns].corr(method='spearman')

import seaborn as sns
import matplotlib.pyplot as plt

sns.set_theme(style="white")
plt.figure(figsize=(8, 6))

heatmap = sns.heatmap(correlation_matrix,
                      annot=True,
                      fmt=".2f",
                      cmap="coolwarm",
                      cbar_kws={'label': 'Spearman Correlation'}
                      )
heatmap.set_title('Spearman Correlation Matrix', fontdict={'fontsize':16}, pad=12)

plt.show()

In [ ]:
# Cell 6: Filter out rows with missing values in key columns
book_missing = books[
        ~(books['description'].isna()) &
        ~(books['num_pages'].isna()) &
        ~(books['average_rating'].isna()) &
        ~(books['published_year'].isna())
    ]

In [ ]:
book_missing.shape

In [ ]:
book_missing['categories'].value_counts().reset_index().sort_values("count", ascending=False)

In [ ]:
# Cell 9: Add a column counting the number of words in each book's description
book_missing['words_in_description'] = book_missing['description'].str.split().str.len()

In [ ]:
book_missing.loc[book_missing['words_in_description'].between(1,4), 'description']

In [ ]:
book_missing.loc[book_missing['words_in_description'].between(5,14), 'description']

In [ ]:
book_missing.loc[book_missing['words_in_description'].between(15,24), 'description']

In [ ]:
book_missing.loc[book_missing['words_in_description'].between(25,34), 'description']

In [ ]:
# Cell 14: Filter books with at least 25 words in the description and show the shape
book_missing_25_words = book_missing[book_missing['words_in_description'] >= 25]
book_missing_25_words.shape

In [ ]:
# Cell 15: Create a new column combining title and subtitle (if available)
book_missing_25_words['title_and_subtitle'] = (
    np.where(
        book_missing_25_words['subtitle'].isna(), book_missing_25_words['title'],
        book_missing_25_words[['title', 'subtitle']].astype(str).agg(': '.join, axis=1)
    )
)

In [ ]:
book_missing_25_words.head(4)

In [ ]:
# Cell 17: Create a new column combining isbn13 and description for tagging
book_missing_25_words['tagged_description'] = book_missing_25_words[['isbn13', 'description']].astype(str).agg(' '.join, axis=1)

In [ ]:
book_missing_25_words.tagged_description

In [ ]:
# Cell 19: Save the cleaned DataFrame to a CSV file, dropping some columns
(
    book_missing_25_words
    .drop(["subtitle", "missing_description", "age_of_book", "words_in_description"], axis=1)
    .to_csv("books_cleaned.csv", index = False)
)